In [17]:
import os 
import sys
import pickle
import folium
import jupyterlab_dash
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import pandas as pd

In [11]:
src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

Data

In [15]:
data_imp = pd.read_pickle('../data/03_processed/all_county_metrics')
updated_county_json = pickle.load(open('../data/03_processed/county_metrics_and_shapefiles.p', 'rb'))
           

Map

In [23]:
aum_map = folium.Map(location=[37, -95], zoom_start=5)
bins = list(data_imp['Absolute Upward Mobility'].quantile([0, 0.25, 0.5, 0.75, 1]))

choro = folium.Choropleth(
    geo_data=updated_county_json,
    data=data_imp,
    columns=['County FIPS Code', 'Absolute Upward Mobility'],
    key_on='properties.GEO_ID',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    nan_fill_color='gray',
    nan_fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Absolute Upward Mobility',
    bins=bins,
    reset=True, 
    highlight=True,
    name='Absolute Upward Mobility'
    
)

choro2 = folium.Choropleth(
    geo_data=updated_county_json,
    data=data_imp,
    columns=['County FIPS Code', 'Target'],
    key_on='properties.GEO_ID',
    fill_color='PuBu',
    fill_opacity=0.5,
    nan_fill_color='gray',
    nan_fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Mobility Outcomes',
    bins=3,
    reset=True, 
    highlight=True,
    name='Child Can Move atleast 1 quartile up'
    )

locations = list(zip(data_imp.lat, data_imp.lon))
popup_content = list(zip(data_imp['County Name'], data_imp['State'], round(data_imp['Absolute Upward Mobility'],1), round(data_imp['Teenage Birth Rate']*100, 1)
                         ))
popups = ['<center> {} County, {} <br>  <b>AUM:</b> {} <br><b>Teen Birth Rate:</b> {}% <br>'.format(
    name, state, aum, share) for (name, state, aum, share) in popup_content]


tooltip = folium.features.GeoJsonTooltip(fields=['NAME', 'State', 'Absolute Upward Mobility', '% Unemployment', '% Teenage Birth Rate', '% Share Between p25 and p75'],
                                         aliases=[
                                             'County', 'State', 'Absolute Upward Mobility', 'Unemployment Rate','Teen Birth Rate', '"Middle Class" Income Share'],
                                         style=('background-color: grey; color: white; font-family:'
                                                'courier new; font-size: 24px; padding: 10px;'),
                                         localize=True)

choro.geojson.add_child(tooltip)

data_imp_copy = data_imp.reset_index()

for idx, row in data_imp_copy.iterrows():
    if row['Absolute Upward Mobility'] >= 60:
        location = locations[idx][0], locations[idx][1]
        marker = folium.Marker(location=location)    
        popup = popups[idx]
        folium.Popup(popup, max_width='150%').add_to(marker)
        icons = folium.Icon(color='green', icon='ok-sign').add_to(marker)
        marker.add_to(choro2)
    elif row['Absolute Upward Mobility'] <= 32:
        location = locations[idx][0], locations[idx][1]
        marker = folium.Marker(location=location)    
        popup = popups[idx]
        folium.Popup(popup, max_width='150%').add_to(marker)
        icons = folium.Icon(color='red').add_to(marker)
        marker.add_to(choro2)
    else: pass

    
choro.add_to(aum_map)
choro2.add_to(aum_map)
folium.LayerControl().add_to(aum_map)



Trying map in plotly

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.fips, z=df.unemp,
                                    colorscale="Viridis", zmin=0, zmax=12,
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Building page layout and adding content

In [27]:
viewer = jupyterlab_dash.AppViewer()
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Link", href="#")),
        dbc.DropdownMenu(
            nav=True,
            in_navbar=True,
            label="Menu",
            children=[
                dbc.DropdownMenuItem("Entry 1"),
                dbc.DropdownMenuItem("Entry 2"),
                dbc.DropdownMenuItem(divider=True),
                dbc.DropdownMenuItem("Entry 3"),
            ],
        ),
    ],
    brand="Demo",
    brand_href="#",
    sticky="top",
)

intro = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H2("Heading"),
                        html.P(
                            """\
Donec id elit non mi porta gravida at eget metus.
Fusce dapibus, tellus ac cursus commodo, tortor mauris condimentum
nibh, ut fermentum massa justo sit amet risus. Etiam porta sem
malesuada magna mollis euismod. Donec sed odio dui. Donec id elit non
mi porta gravida at eget metus. Fusce dapibus, tellus ac cursus
commodo, tortor mauris condimentum nibh, ut fermentum massa justo sit
amet risus. Etiam porta sem malesuada magna mollis euismod. Donec sed
odio dui."""
                        ),
                        dbc.Button("View details", color="secondary"),
                    ],
                    md=4,
                )
            ]
        )
    ]
)
            
map_plot = dbc.Container(
    [
         dbc.Row(
             [
                html.H2("Graph")
#                 dcc.Graph(
#                             figure=aum_map ## Can I make this part the map I've already programmed? 
#                         )
                    ]
                )
            ]
        )

row = html.Div(
    [
        dbc.Row(dbc.Col(html.Div("A single column"))),
        dbc.Row(
            [
                dbc.Col(html.Div("One of three columns")),
                dbc.Col(html.Div("One of three columns")),
                dbc.Col(html.Div("One of three columns")),
            ]
        ),
    ]
)

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div([navbar, row, intro, map_plot])

viewer.show(app)